In [2]:
%autoreload 2
import sys
sys.path.append('C:\\code\\qdc2')
import numpy as np 
import matplotlib.pyplot as plt
from qdc.grating.grating_sim import GratingSim1D

In [21]:
sim = GratingSim1D(
    Nx          = 2**16,      # grid points
    Lx          = 10e-3,       # 8 mm
    wl0         = 808e-9,     # 808 nm blaze
    Dwl         = 80e-9,     # ±50 nm span
    N_wl        = 51,         # 11 wavelengths  (odd!)
    waist       = 300e-6,      # 40 µm beam waist
    x0          = 0,
    blaze_angle = 0.1,       # blaze angle [rad]
    f           = 0.2,         # lens focal length [m]
    spectrum='flat', 
)

# Numerical patterns (as before)
x_det, I_class_num = sim.classical_pattern()
_,     I_spdc_num  = sim.spdc_pattern()

# Analytic predictions
x_det, I_class_anal = sim.analytical_pattern(is_spdc=False, n_side=8)
_,     I_spdc_anal  = sim.analytical_pattern(is_spdc=True, n_side=8)

# Convert x_det to diffraction order axis
m_vals = sim.diffraction_orders(x_det)


fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(m_vals, I_class_num,  'b-', label="Numerical Classical")
ax.plot(m_vals, I_class_anal, 'b--', label="Analytic Classical")
ax.plot(m_vals, I_spdc_num,   'r-', label="Numerical SPDC")
ax.plot(m_vals, I_spdc_anal,  'r--', label="Analytic SPDC")
ax.set_yscale('log')
ax.set_xlabel("diffraction order $m$")
ax.set_ylabel("normalized intensity (log)")
ax.set_ylim(1e-6, 2e3)
ax.set_xlim(-3.5, 5.5)
ax.set_title(f"{sim.N_wl} wavelengths")
ax.legend()
fig.tight_layout()
fig.show()

In [9]:
print((x_det[1] - x_det[0])*I_class_num.sum())
print((x_det[1] - x_det[0])*I_spdc_num.sum())
print(I_spdc_num.max())
print(I_class_num.max())

0.9999967262587829
1.0000002595434
270.1878398580031
526.6634206581928


In [7]:
from qdc.grating.grating_sim import gaussian, blazed_phase

if True:
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(sim.grating_phase, '--.')
    # ax.set_xlim([0, 1400])
    fig.show()
 
if True:
    E = gaussian(sim.x, sim.waist, sim.x0).astype(np.complex128)  
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot((np.abs(E))**2)
    fig.show()

In [118]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve

# Physical parameters
d = 10e-6               # grating period [m]
k0 = 2 * np.pi / d  # Blaze toward m=1

delta_k = 0             # small phase offset [rad/m]
w = 1e-3                # beam waist [m]
f = 0.2                 # focal length [m]
lambda0 = 800e-9        # wavelength [m]
k = 2 * np.pi / lambda0

# Define x' axis in focal plane
x_prime = np.linspace(-5e-3, 5e-3, 5000)  # in meters
kx = (k / f) * x_prime

# Fourier transform of motif over [0,d)
def motif_ft(kx, k0, delta_k, d):
    return d * np.sinc((kx - k0 - delta_k) * d / (2 * np.pi))

# Fourier transform of Gaussian beam
def gauss_ft(kx, w):
    return np.sqrt(np.pi) * w * np.exp(- (kx**2) * w**2 / 4)

# Dirac comb in Fourier domain (sampled)
def dirac_comb(kx, d):
    # Generate spikes at multiples of 2pi/d within the range of kx
    km_values = 2 * np.pi * np.arange(-50, 51) / d
    comb = np.zeros_like(kx)
    for km in km_values:
        comb += np.exp(-((kx - km)**2) / (2 * (2e3)**2))  # narrow Gaussian for numerical delta
    return (2 * np.pi / d) * comb

# Combined field spectrum
motif = motif_ft(kx, k0, delta_k, d)
gauss = gauss_ft(kx, w)
comb = dirac_comb(kx, d)
spectrum = comb * motif
spectrum_convolved = convolve(spectrum, gauss, mode='same') * (kx[1] - kx[0])
intensity = np.abs(spectrum_convolved)**2


import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(x_prime * 1e3, intensity / np.max(intensity), lw=2)
plt.xlabel("Position $x'$ [mm]")
plt.ylabel("Normalized Intensity")
plt.yscale('log')
plt.title("Classical Focal-Plane Intensity after Blazed Grating")
plt.grid(True)
plt.tight_layout()
plt.show()